In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from scipy.optimize import fsolve
from scipy import stats

#TODO: This file has problems

In [2]:
def contraction(params, x, p):
    # beta and x are kind of parameters. x is the empirical distribution of x?
    rho = np.clip(params[-1],-1,1)
    k = int(x.shape[1]/2)
    util1 = np.dot(x[:, 0:k], params[0:k]) + params[-3]*p[:,1]
    util2 = np.dot(x[:, k:2*k], params[k:2*k]) + params[-2]*p[:,0]
    p = np.array([np.exp(util1)/(1+np.exp(util1)),
                    np.exp(util2)/(1+np.exp(util2))])
    
    #summarize prob
    prob01 = (1 - p[0, :])*(p[1, :]) 
    prob10 = (p[0, :])*(1-p[1, :]) 
    prob00 = (1-p[0, :])*(1-p[1, :]) 
    prob11 = (p[0, :])*(p[1, :]) 
    
    #introduce the copula
    prob01_rho = prob01*(1+rho*prob10)
    prob10_rho = prob10*(1+rho*prob01)
    prob00_rho = prob00*(1+rho*prob11)
    prob11_rho = prob11*(1+rho*prob00)
    
    return np.array([prob01,prob10,prob00,prob11])


def contraction_map(betas, x, p):
    """final result is beliefs of firm1/firm2"""
    full_p = None
    for i in range(50):
        full_p = contraction(betas, x, p)
        
        #use bayes rule to figure out full pr?
        p1 = full_p[1] + full_p[3]
        p2 = full_p[0] + full_p[3]
        p = np.array([p1,p2]).transpose()
        
    return p

#actually caclualte an equilibrium in this game
N=1000
ps = np.random.randint(2, size=(N,2))
rho = .5
betas = np.array([1, 2, -2, -1, rho])

#set up xs
xs = np.random.normal(scale=2, size=(N,2)) #assumes xs are independent? could use a copula?
xs = pd.DataFrame(xs,columns = ['x11','x12'])
xs = np.array(xs)

result_ps = contraction_map(betas,xs,ps)

In [3]:
def conditional_cdf(x1,x2,rho):
    u = stats.logistic.cdf(x1)
    v = stats.logistic.cdf(x2)
    joint = u*v*(1+rho*(1-v)*(1-u))
    return joint/v

func = lambda x1 : .6 - conditional_cdf(x1,2,rho)

e1 = fsolve(func, [.3])

In [4]:
k = int(xs.shape[1]/2)
e1s = np.random.logistic(0, 1, N )
e2s = []

#draw from e2 conditional on e1
for e1 in e1s:
    inverse_cdf = np.random.uniform(0,1,1)
    func = lambda x1 : inverse_cdf - conditional_cdf(x1,e1,rho)
    e2 = fsolve(func, [0.0])
    e2s.append(e2[0])
e2s = np.array(e2s)

#simulate the copula?
y1 = 1*(np.dot(xs[:, 0:k], betas[0:k])  + betas[-3]*result_ps[:,1] + e1s >= 0)
y2 = 1*(np.dot(xs[:, k:2*k], betas[k:2*k]) + betas[-2]*result_ps[:,0] + e2s >= 0)
ys = np.array([y1,y2]).transpose()

print(result_ps[(xs[:,1] > 0)].mean(axis=0))
print(ys[ (xs[:,1] > 0) ].mean(axis=0))
print('----')
print(result_ps[(xs[:,0] > 0)].mean(axis=0))
print(ys[ (xs[:,0] > 0) ].mean(axis=0))

[0.29395248 0.82408685]
[0.27606178 0.7953668 ]
----
[0.60335708 0.48591167]
[0.58928571 0.47420635]


In [5]:
result_df = np.concatenate( (ys,xs ) ,axis=1)
result_df = pd.DataFrame(result_df, columns=['y1','y2','x11','x12'])
print(result_df)
result_df.to_csv('monte_carlo.csv')

      y1   y2       x11       x12
0    0.0  1.0  2.027584 -0.490070
1    0.0  1.0 -2.802596  1.099926
2    0.0  1.0 -1.028362  2.264912
3    0.0  1.0  0.501097  1.502984
4    0.0  1.0 -1.608895  3.902755
..   ...  ...       ...       ...
995  0.0  1.0  1.826458  2.698067
996  1.0  0.0  1.962563 -1.620564
997  0.0  0.0 -0.535656 -3.408290
998  0.0  1.0 -4.044969  1.586089
999  1.0  0.0  0.417605 -0.164867

[1000 rows x 4 columns]


In [6]:
class BayesNashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        
        p = self.endog.mean()
        p = contraction_map(params,self.exog,p)
        
        likelihood = contraction(params,self.exog,p).transpose()
        ll = self.endog*np.log(likelihood) + (1-self.endog)*np.log(1-likelihood)
        return -1*ll.sum(axis=1)
        
        
    
    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        start_params = np.ones(self.exog.shape[1]+2)
        return super(BayesNashLogit, self).fit(start_params=start_params,**kwds)

    
    
class BayesNashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)

        p = self.endog
        p = contraction_map(params, self.exog, p)

        likelihood = contraction(params, self.exog, p)
        ll = self.endog*np.log(likelihood) + \
            (1-self.endog)*np.log(1-likelihood)
        return -1*ll.sum(axis=1)


        
        
N = result_df.shape[0]
ys = result_df[['y1','y2']]
xs = result_df[['x11' ,'x12']]
print(xs.shape)

model = BayesNashLogit(ys,xs)
#model.loglikeobs(np.ones(xs.shape[1]+2))
model_fit = model.fit(xtol=1e-12,ftol=1e-12)
print(model_fit.summary(xname=["b11","d1","b12","d2"]))


(1000, 2)


IndexError: index -3 is out of bounds for axis 0 with size 2

In [ ]:
#attempt at standard errors...
H = np.linalg.inv(model.hessian(model_fit.params)/N)
diag = np.diagonal(np.linalg.inv(model.hessian(model_fit.params)))
print(np.sqrt(-1*diag))

In [ ]:
# Modify the variables
entry_data = pd.read_csv("merged_entry.csv")
entry_data.loc[entry_data['HD'] > 0.0, 'HD_1'] = 1
entry_data.loc[entry_data['LO'] > 0.0, 'LO_1'] = 1
entry_data['income_2'] = np.log(1 + entry_data['income_per_capita'])
entry_data['population_2'] = np.log(1 + entry_data['population'])
entry_data['under44_1_2'] = np.log(1 + entry_data['under44_1'])
entry_data['under44_2_2'] = np.log(1 + entry_data['under44_2'])
entry_data['under44_3_2'] = np.log(1 + entry_data['under44_3'])
entry_data['older65_1_2'] = np.log(1 + entry_data['older65_1'])
entry_data['older65_2_2'] = np.log(1 + entry_data['older_65_2'])
entry_data = entry_data.fillna(0)

In [ ]:
# Fit the model
N = entry_data.shape[0]
x = entry_data[['income_2','population_2','income_2','population_2']].copy()
y = entry_data[['HD_1','LO_1']]

model = BayesNashLogit(y,x).fit(xtol=1e-6,ftol=1e-6)
model.summary()